In [1]:
import acquire
import prepare
import scrape

import pandas as pd
import matplotlib as plt
import numpy as np



In [24]:
def github_urls():
    '''
    This function scrapes all of the evironmental urls from
    the github search page and returns a list of urls.
    '''
    # The base url for the main Codeup blog page
    url = 'https://github.com/search?o=desc&p=1&q=environmental&s=&type=Repositories'
    
    # Make request and soup object using helper
    soup = acquire.make_soup(url)
    
    # Create a list of the anchor elements that hold the urls.
    urls_list = soup.find_all('a', class_='v-align-middle')
    # for each url in the find all list get just the 'href' link
    urls = {link.get('href') for link in urls_list}
    # make a list of these urls
    urls = list(urls)
    return urls
# this gets 1st 10 urls, will need to get next 10 pages


In [25]:
url_list = github_urls()
url_list

['/publiclab/leaflet-environmental-layers',
 '/rbray89/EnvironmentalVisualEnhancements',
 '/emoncms/emoncms',
 '/sqlcollaborative/dbachecks',
 '/ejgertz/EMWA',
 '/karolpiczak/ESC-50',
 '/kizniche/Mycodo',
 '/karolpiczak/EARS',
 '/Genetic-Malware/Ebowla',
 '/rockita/Environmental_Intelligence']

In [26]:
def get_github(url_list, cached=False):
    '''
    This function with default cached == False does a fresh scrape of inshort pages with topics 
    business, sports, technology, and entertainment and writes the returned df to a json file.
    cached == True returns a df read in from a json file.
    '''
    # option to read in a json file instead of scrape for df
    if cached == True:
        df = pd.read_json('readmes.json')
        
    # cached == False completes a fresh scrape for df    
    else:
    
        # Set base_url that will be used in get request
        base_url = 'https://github.com'
        
        # List of full url needed to get readme info
        readme_url_list = []
        for url in url_list:
            full_url = base_url + url
            readme_url_list.append(full_url)
        
        # Create an empty list, readmes, to hold our dictionaries
        readmes = []

        for readme_url in readme_url_list:
            # Make request and soup object using helper
            soup = make_soup(readme_url)

            # Save the text in each readme to variable text
            content = soup.find(xxxx).text

            # Save the first language in each readme to variable text
            language = soup.find(xxxx).text

            # anything else useful on the page?
            
            # Create a dictionary holding the title and content for each blog
            readme = {'language': language, 'content': content}

            # Add each dictionary to the articles list of dictionaries
            readmes.append(readme)
            
        # convert our list of dictionaries to a df
        df = pd.DataFrame(readmes)

        # Write df to a json file for faster access
        df.to_json('readmes.json')

    
    return df

In [27]:
readme_test = get_github(url_list, cached=False)
readme_test

['https://github.com/publiclab/leaflet-environmental-layers',
 'https://github.com/rbray89/EnvironmentalVisualEnhancements',
 'https://github.com/emoncms/emoncms',
 'https://github.com/sqlcollaborative/dbachecks',
 'https://github.com/ejgertz/EMWA',
 'https://github.com/karolpiczak/ESC-50',
 'https://github.com/kizniche/Mycodo',
 'https://github.com/karolpiczak/EARS',
 'https://github.com/Genetic-Malware/Ebowla',
 'https://github.com/rockita/Environmental_Intelligence']